# ToS Mixedbread Embedding Creation

State-of-the-art sentence embeddings from mixedbread.ai. 

https://huggingface.co/mixedbread-ai/mxbai-embed-large-v1.

In [21]:
# Packages
from sentence_transformers import SentenceTransformer
# NLTK for sentence tokenization
import nltk
nltk.download('punkt')
# Torch to move to GPU
import torch
import os
import pandas as pd
import time

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ijyli\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [22]:
# Flag for if this is a sample run or not
sample_run = True

## Load Model

In [23]:
model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")
# Move to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)
# Print model
print(model)
# Print device
print(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)
cpu


## Function to Get Sentences and Embeddings from Doc

In [24]:
def get_doc_sentences_embeddings(filename):
    '''
    Get sentence embeddings from a document.
    '''
    # Read file
    with open('../Text Data/Terms of Service Corpus/text/' + filename, 'r', encoding='utf-8') as f:
        doc = f.read()
    # Strip all unicode characters
    doc = ''.join([char if ord(char) < 128 else ' ' for char in doc])
    # Strip lines that do not contain a period, exclamation point, or question mark, and at least one letter
    doc = '\n'.join([line for line in doc.split('\n') if ('.' in line or '!' in line or '?' in line) and any([char.isalpha() for char in line])])
    # Parse company name by taking the first part of filename before underscore
    company_name = filename.split('_')[0]
    # Sentence tokenize the document
    sentences = nltk.sent_tokenize(doc)
    # Append company name to the beginning of each sentence
    sentences = [company_name + ' ' + sentence for sentence in sentences]
    # Encode sentences and return
    embeddings = model.encode(sentences)
    return sentences, embeddings
    

## Get List of files

In [25]:
filenames = os.listdir('../Text Data/Terms of Service Corpus/text')
print('length of filenames:', len(filenames))
print(filenames[:5])

length of filenames: 9491
['115media_CookiePolicy.txt', '115media_privacy.txt', '115media_Terms.txt', '11soundboards_PrivacyPolicy.txt', '123tvlive_PrivacyPolicy.txt']


## Encode Documents and Create Parquet Files

In [26]:
def create_parquet(filename):
    '''
    Save parquet file of sentences and embeddings for a document.
    '''
    # Get sentences and embeddings
    # Start timer
    start = time.time()
    sentences, embeddings = get_doc_sentences_embeddings(filename)
    # End timer
    end = time.time()
    print('Time to get embeddings:', end - start)
    # New timer
    start = time.time()
    # Create dataframe
    df = pd.DataFrame()
    # Sentences are a column
    df['sentence'] = sentences
    # Add embeddings array on to the dataframe
    df = pd.concat([df, pd.DataFrame(embeddings)], axis=1)
    # Set column names
    column_names = ['sentence'] + [f'embed_element_{i}' for i in range(embeddings.shape[1])]
    df.columns = column_names
    # Save to parquet
    df.to_parquet('../Text Data Embeddings/' + filename.split('.')[0] + '.parquet')
    # End timer
    end = time.time()
    print('Time to save parquet:', end - start)

In [27]:
# If this is a sample run, run for one filename + the largest file, otherwise, run for all
if sample_run:
    create_parquet(filenames[0])
    #create_parquet('Honeywell_CookieNotice.txt')

Time to get embeddings: 9.69528317451477
Time to save parquet: 0.40622687339782715


In [28]:
# Load a file to check
test_df = pd.read_parquet('../Text Data Embeddings/' + filenames[0].split('.')[0] + '.parquet')
test_df.head()

,sentence,embed_element_0,embed_element_1,embed_element_2,embed_element_3,embed_element_4,embed_element_5,embed_element_6,embed_element_7,embed_element_8,...,embed_element_1014,embed_element_1015,embed_element_1016,embed_element_1017,embed_element_1018,embed_element_1019,embed_element_1020,embed_element_1021,embed_element_1022,embed_element_1023
0,115media Careers &amp.,0.081622,0.186236,-0.403659,0.188761,0.116395,-0.178390,-0.186043,-0.127867,0.417745,...,0.916923,-0.275322,-0.153453,-0.369527,0.779784,0.601135,0.300785,-0.366489,-0.805873,0.564936
1,115media Why Work for 1105?,0.402588,0.343286,-0.282753,0.840287,0.062783,-0.187640,0.449536,-0.195319,0.155475,...,0.765590,-0.443140,-0.112778,0.208237,0.521470,0.942637,0.328892,-0.428862,-0.576944,0.530157
2,115media Apply Today!,-0.223923,0.459062,-0.686977,0.469076,-0.083358,0.541975,-0.136509,0.417895,0.637240,...,0.907129,-0.562752,-0.057301,-0.782100,0.866764,0.895059,0.267527,-0.193847,-0.860717,0.768427
3,115media How Can We Help?,0.216342,0.515327,-1.257966,0.625449,0.365142,-0.079858,0.517099,0.627124,0.538878,...,0.973817,-0.428779,-0.197326,-0.630904,1.070016,1.036640,0.317606,-0.424600,-0.603931,0.496899
4,115media 1105 Media uses industry standard coo...,0.155285,0.671658,0.072046,0.605800,-0.004579,-0.107331,0.256348,-0.583437,0.760930,...,0.471236,-0.329914,-0.100052,-0.264092,1.008361,0.510139,-0.175097,-0.135984,-0.023450,1.076122


In [29]:
# Print all values of sentence column
for value in test_df['sentence'].values:
    print(value)

115media Careers &amp.
115media Why Work for 1105?
115media Apply Today!
115media How Can We Help?
115media 1105 Media uses industry standard cookie technology (examples below) in order to maximize customer experience and operational efficiency.&nbsp.
115media When you visit one of 1105 Media's websites, we may send you a cookie.
115media A cookie is a small file which is placed on your computer or device.
115media These cookies are essential for the operation of our websites.
115media Without the use of these cookies, parts of our websites would not function.
115media For example, we use cookies to help us identify which of our readers have previously registered in order to access premium content on our websites.
115media We use these types of cookies to monitor our websites' performance and how users may interact with it.
115media These cookies provide us with information that helps us provide better products to our users and also to identify any areas that may need maintenance.
115m

In [30]:
# Load Honeywell file to check
test_df = pd.read_parquet('../Text Data Embeddings/Honeywell_CookieNotice.parquet')
test_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../Text Data Embeddings/Honeywell_CookieNotice.parquet'